In [1]:
import lincs
import liblincs

In [2]:
def walk(output_file, node_name, node, parent_node=None, depth=0):
    indent = "    " * depth
    node_type = type(node)
    node_type_name = node_type.__name__

    # Don't show objects of these types
    if node_type_name in [
        "ModuleSpec", "SourceFileLoader", "ExtensionFileLoader",
    ]:
        return
    
    # Don't show special attributes
    if node_name.startswith("__") and node_name.endswith("__") and node_name not in ["__init__"]:
        return

    # Defensive: never recurse too deep
    if depth > 4:
        output_file.write(f"{indent} {node_name}: {node_type_name}  UNFINISHED\n")
        return

    output_file.write(f"{indent}{node_name}: {node_type_name}\n")

    # Don't recurse into our enums: enum values have an attribute pointing to its type
    if node_type == parent_node:
        return

    # Don't recurse into objects of these types
    if node_type_name in [
        "wrapper_descriptor", "getset_descriptor", "method_descriptor", "member_descriptor",
        "property", "function", "method", "builtin_function_or_method",
        "str", "bool", "int", "float", "NoneType",
        "list", "dict", "tuple",
    ]:
        return

    for child_name in sorted(dir(node)):
        try:
            child_node = getattr(node, child_name)
        except AttributeError:
            # Don't get stuck on attributes listed in 'dir' but not actually there
            pass
        else:
            walk(output_file, child_name, child_node, node, depth + 1)


with open("lincs.txt", "w") as f:
    walk(f, "lincs", lincs)


with open("liblincs.txt", "w") as f:
    walk(f, "liblincs", liblincs)